In [1]:
#get document frequency for terms in the same article/instance; and get each sentence's BOW; and get each question's BOW;
#get each sentence's length; get each article's average length of sentences
import csv
import json
import copy
import os
import nltk
from math import log
from collections import defaultdict, Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial.distance import cosine as cos_distance
from sklearn.decomposition import TruncatedSVD
from nltk.tag import StanfordNERTagger
def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma
def input_data():
    base_path = os.path.join('data/')
    train_file = base_path + 'QA_train.json'
    train_data = json.load(open(train_file))
    test_file = base_path + 'QA_test.json'
    test_data = json.load(open(test_file))
    dev_file = base_path + 'QA_dev.json'
    dev_data = json.load(open(dev_file))

    return train_data,test_data,dev_data
def transform_text(text):
    text = nltk.word_tokenize(text)
    text = [lemmatize(word.lower()) for word in text]
    result = []
    for word in text:
        if word not in stopwords and word not in punctuations:
            result.append(word)
    return result
def get_Docfrequency_SentenceBOW(dataset):
    #save dics, each dictionary contains document frequencies for all terms in the same article
    question_list = []
    #save lists, each list represent an article, saving sentences' bow
    total_sentence_bow = []
    #save lists, each list represent an article, saving questions' bow
    total_question_bow = []
    #save lists, each list represent all sentences' lengthes.
    sent_lengthes = []
    #save a list, each item represents the average length of sentences
    avg_lengthes = []
    #
    answer_id = []
    
    for article in dataset:
        #Docfrequency
        article_dic = defaultdict(list)
        keyterms = [] #save all distinct terms in questions
        
        #SentenceBOW
        bow_list = []
        
        #QuestionBOW
        que_list = []
        
        #SentenceLength
        sent_len = []
        
        #TotalLength
        total_len = 0
        
        qas = article['qa']
        sentences = article['sentences']
        for qa in qas:
            question = qa['question']
            newquestion = transform_text(question)
            #QuestionBOW
            que_list.append(get_BOW(newquestion))
            
            keyterms.extend(newquestion)
        keyterms = set(keyterms)
        
        #save sentences' BOW in list sen_BOW
        sen_words = []
        for sent in sentences:
            sent = transform_text(sent)
            #Docfrequency
            sen_words.append(sent)
            
            #SentenceBOW
            bow_list.append(get_BOW(sent))
            
            #SentenceLength
            sent_len.append(len(sent))
            
            #TotalLength
            total_len += len(sent)
            
        
        #calculate doc frequency    
        for term in keyterms:
            for i,bow in enumerate(sen_words):
                if term in bow:
                    article_dic[term].append(i)
                    
        #Docfrequency
        question_list.append(article_dic)
        #SentenceBOW
        total_sentence_bow.append(bow_list)
        #QuestionBOW
        total_question_bow.append(que_list)
        #SentenceLength
        sent_lengthes.append(sent_len)
        #AverageLength
        avg_lengthes.append(float(total_len)/len(sentences))

        
    return question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes

train,test,dev = input_data()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
punctuations = [',','\'\'','?','\'','.','%','(',')',';','``']
question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes = get_Docfrequency_SentenceBOW(test)

In [2]:
import math
import operator
def BM25_n(articles_index,k1,k2,b,n):
    total_queries = len(total_question_bow[articles_index])
    count = 0
    correct_id = []
    correct_id_weight = []
    for index in range(len(total_question_bow[articles_index])):
        poss_results = find_max_n_sentences(articles_index,index,k1,k2,b,n)
        guess_ids = []
        weights = []
        for sent in poss_results:
            guess_ids.append(sent[0])
            weights.append(sent[1])
        correct_id.append(guess_ids)
        correct_id_weight.append(weights)
    return correct_id,correct_id_weight

def find_max_n_sentences(articles_index,index,k1,k2,b,n):
    query_dict = total_question_bow[articles_index][index]
    scores = {}
    for index in range(len(total_sentence_bow[articles_index])):     
        score = 0  
        sentence_dict = total_sentence_bow[articles_index][index]
        for word in query_dict:
            document_fre_list = question_list[articles_index].get(word,None)
            N = len(total_sentence_bow[articles_index])
            n_qi = 0
            if document_fre_list != None:
                n_qi = len(document_fre_list)
            else:
                n_qi = 0
            fi = sentence_dict.get(word,0)
            qfi = query_dict.get(word,0)
            dl = sent_lengthes[articles_index][index]
            avgdl = avg_lengthes[articles_index]
            
            K = k1*(1-b+b*(float(dl)/avgdl)) 
            W = math.log((N-n_qi+0.5)/(n_qi+0.5))
            R = (fi*(k1+1))/(fi+K)*qfi*(k2+1)/(qfi+k2)
            score += W*R
        scores[index] = score
    scores = sorted(scores.items(), key=operator.itemgetter(1),reverse=True)[:n]
    return scores

In [17]:
k1_list = [0.78]
k2_list = [0]
b_list = [0.5]

result_sentences = []
result_sentences_weight = []
test_length = len(test)
for k1 in k1_list:
    for k2 in k2_list:
        for b in b_list:
            for i in range(0,test_length):
                correct_id,correct_id_weight = BM25_n(i,k1,k2,b,4)
                result_sentences.append(correct_id)
                result_sentences_weight.append(correct_id_weight)

In [18]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import re
import sys

location_list = ['country','county','district','city']
number_list = ['version','size','msa','far','much','ration','time','many','population','large','percent','average','day','decade','big','long']
time_list = ['january','february','march','april','may','june','july','august','september','october','november','december','hours','minutes','miles','inches','foot','years','seasons','half']
name_list = ['name','center','president','denominations','denomination','film','broadcaster','pitcher','commentator']
year_list = ['years','year','era']

def transfer_pos_question(pos):
    new_pos = []
    for (word,wtype) in pos:
        if word.lower() == 'what' or word.lower() == 'what\'s':
            new_pos.append((word,'WHAT'))
        elif word.lower() == 'do' or word.lower() == 'does' or word.lower() == 'did':
            new_pos.append((word,'DO'))
        elif word.lower() in number_list:
            new_pos.append((word,'TIME'))
        elif word.lower() in year_list:
            new_pos.append((word,'YEAR'))
        elif word.lower() == 'is' or word.lower() == 'was' or word.lower() == 'are' or word.lower() == 'were' or word.lower() == 'be':
            new_pos.append((word,'BE'))
        elif word.lower() in location_list:
            new_pos.append((word,'LOC'))
        elif word.lower() == 'when':
            new_pos.append((word,'WHEN'))
        elif word.lower() == 'where':
            new_pos.append((word,'WHERE'))
        elif word.lower() == 'can':
            new_pos.append((word,'CAN'))
        elif word.lower() == 'how':
            new_pos.append((word,'HOW'))
        elif word.lower() == 'who' or word.lower() == 'whom' or word.lower() == 'whose'  or word.lower() == 'whos':
            new_pos.append((word,'WHO'))
        elif word.lower() == 'which':
            new_pos.append((word,'WHICH'))
        elif word.lower() in name_list:
            new_pos.append((word,'NAME'))
        elif word.lower() == 'define':
            new_pos.append((word,'DEFINE'))
        elif word.lower() == 'should':
            new_pos.append((word,'SHOULD'))
        elif word.lower() == 'why' or word.lower() == 'wy':
            new_pos.append((word,'WHY'))
        else:
            new_pos.append((word,wtype))
    return new_pos

def get_continuous_chunks(text):
    t = copy.deepcopy(text)
    pos = pos_tag(nltk.word_tokenize(t))
    pos = transfer_pos_question(pos)
    #print pos
    grammar = r"""
                WHAT: 
                    {<WHAT>}
                    {<WHICH>}
                    {<DEFINE>}
                WHO:
                    {<WHO>}
                    {<WHAT><BE>?<DT>?<JJ|RB>*<NAME>}
                    {<WHAT><JJ|RB>*<NN>+<NAME>}
                NUMBER:
                    {<WHICH><TIME>}
                    {<HOW><TIME>}
                    {<WHAT><BE>?<DT>?<JJ>?<NN>*<JJ>?<TIME>}
                WHEN:
                    {<WHICH><YEAR>}
                    {<WHAT><BE>?<DT>?<JJ>?<NN>*<JJ>?<YEAR>}
                    {<WHEN>}
                WHERE:
                    {<WHERE>}
                    {<WHAT><LOC>}
                    {<WHAT><BE>?<DT>?<RBS>?<JJ>?<LOC>}
                HOW:
                    {<CAN>}
                    {<DO>}
                    {<SHOULD>}
                    {<WHY>}
                    {<HOW>}

                """
    cp = nltk.RegexpParser(grammar) 
    result = []
    tree = cp.parse(pos)
    #record the position of pos
    #print pos
    flag = 0
    for subtree in tree.subtrees():
        if subtree.label() != 'S':
            phrase = u''
            for word,pos in subtree.leaves():
                if word == ',':
                    phrase = phrase + word
                else:
                    phrase = phrase + u' '
                    phrase = phrase + word
            result.append((subtree.label(),phrase[1:]))         
    return result

In [19]:
def transfer_pos_sentence(pos):
    new_pos = []
    
    for (word,wtype) in pos:
        time_pattern = re.compile(r'[0-9]{4}s?')
        if word.lower() == 'and' or word.lower() == 'or':
            new_pos.append((word,'POSICC'))
        elif word.lower() == 'with':
            new_pos.append((word,'WITH'))
        elif word.lower() == 'a' or word.lower() == 'an':
            new_pos.append((word,'A'))
        elif word == '"':
            new_pos.append((word,'"'))
        elif word == 'minutes' or word == 'March':
            new_pos.append((word,'TIME'))
        elif time_pattern.search(word):
            new_pos.append((word,'TIME'))
        elif word == 'around':
            new_pos.append((word,'AROUND'))
        elif word == 'mm':
            new_pos.append((word,'CD')) 
        else:
            new_pos.append((word,wtype))
    return new_pos

def get_continuous_chunks_sentence(text,texttype):
    t = copy.deepcopy(text)
    pos =  pos_tag(nltk.word_tokenize(t))
    if texttype==0:
        #WHAT
        pos = transfer_pos_sentence(pos)
        grammar = r"""
                    J:
                        {<JJ.*><VBN>}
                        {<JJ.*><POSICC><JJ.*>}   
                        {<JJ.*>+}
                        {<NN.*><POS>}
                    N:
                        {<CD>+<NN.*>}
                        {<A>?<NN.*>?<J>?<NN.*>+}
                        <\">{<A>?<J>?<NN.*>+}<\">
                    COMBON:
                        {(<N><,>)*<N><,>?<POSICC><N>}
                    NWC:
                        {<N><WITH><COMBON>}
                    """
    elif texttype==1:
        #when
        pos = transfer_pos_sentence(pos)
        #print pos
        grammar = r"""
                    C:
                        {<CD><TIME>}
                        {<CD><\.><CD>}
                        {<RB><CD>+}   
                        {<JJ><CD>+}
                        {<AROUND><CD>+}
                        {<RB><TIME>+}   
                        {<JJ><TIME>+}
                        {<AROUND><TIME>+}
                        {<TIME>+}
                        {<CD>+}      
                    COMBOC:
                        {<C><POSICC><C>}
                    """
    elif texttype==2:
        #when
        pos = transfer_pos_sentence(pos)
        #print pos
        grammar = r"""
                    H:
                        {<RB><JJ><IN>?}
                    """
    cp = nltk.RegexpParser(grammar) 
    result = []
    poss = copy.deepcopy(pos)
    tree = cp.parse(pos)
    #record the position of pos
    flag = 0
    for i in range (len(tree)):   
        if type(tree[i]) != tuple:
            subtree = tree[i]
            if texttype==0 and subtree.label() != 'S':
                phrase = u''
                for word,pos in subtree.leaves():
                    if word == ',':
                        phrase = phrase + word
                    else:
                        phrase = phrase + u' '
                        phrase = phrase + word
                result.append((subtree.label(),phrase[1:]))
                #print subtree.label(),phrase
            elif subtree.label() != 'S':
                phrase = u' '.join([word for word,pos in subtree.leaves()])
                result.append((subtree.label(),phrase))      
    return poss,result

In [20]:
'''
NER
'''
time_word = [
    'one','two','three','four','five','six','seven','eight','nine',
    'January','February','March','April','May','June','July','August','September','October','November','December',
    'million','billion',
    'minutes','hours','years','times',
    'mm','miles','inches','foot','feet',
    'late','early','around','over',
    'persons','seasons','square',
    'spring','summer','fall','autumn','winter'
]

location_word = [
    'southwest','southeast','northwest','northeast'
]

conjunction_word = ['and','of']

from nltk.tag import StanfordNERTagger
import re
def input_NER():
    stanford_dir = os.path.join('stanford-ner-2016-10-31')
    jarfile = os.path.join(stanford_dir,'stanford-ner.jar')
    modelfile = os.path.join(stanford_dir,'classifiers/english.all.3class.distsim.crf.ser.gz')
    return modelfile,jarfile
model,jar = input_NER()
st = StanfordNERTagger(model,jar)

def analyse_NER(ner_sentences):
    result_sentences = []
    for ner_sentence in ner_sentences:
        result_sentence = []
        perv_type = u'O'
        word = u''
        conjunction = u''
        conjunc_flag = False
        for index,(entity,etype) in enumerate(ner_sentence):
            if perv_type == u'O' and etype != u'O':              
                perv_type = etype
                word = entity + u' '
            elif word != u'':
                if etype == u'O':
                    if entity not in conjunction_word:
                        result_sentence.append((word[:-1],perv_type))
                        word = u''
                        perv_type = u'O'
                        if conjunction != u'':
                            conjunction = u''
                            conjunc_flag = False
                    else:
                        if conjunction != u'':
                            conjunction = u''
                            conjunc_flag = False
                        else:
                            conjunction = entity
                            conjunc_flag = True
                elif etype != perv_type:
                    result_sentence.append((word[:-1],perv_type))
                    word = entity + u' '
                    perv_type = etype
                    conjunction = u''
                    conjunc_flag = False
                elif etype == perv_type:
                    if conjunc_flag:
                        if conjunction == u',':
                            word = word[:-1] + conjunction + u' ' + entity + u' '
                        else:
                            word = word + conjunction + u' ' + entity + u' '
                        conjunction = u''
                        conjunc_flag = False
                    else:
                        if entity in ['%'] or word == u'$ ':
                            word = word[:-1] + entity + u' '
                        else:
                            word = word + entity + u' '
        if word != u'':
            result_sentence.append((word[:-1],perv_type))
        result_sentences.append(result_sentence)      
    return result_sentences

def parse_NER(ner_sentences):
    pattern_number = re.compile(r'([0-9]+|\%|\$)')
    year_number = re.compile(r'([0-9]{4}s?)')
    result_sentences = []
    for ner_sentence in ner_sentences:
        result_sentence = []
        for index,(entity,etype) in enumerate(ner_sentence):
            if entity != u'':
                entity.replace(u'\u2013',u'-')
                entity.replace(u'\u2014',u'-')
                entity.replace(u'\u2212',u'-')
                entity.replace(u'\u2044',u'%')
                if etype == u'O':
                    if year_number.search(entity):
                        result_sentence.append((entity,u'YEAR'))
                    elif pattern_number.search(entity) or entity in time_word:
                        result_sentence.append((entity,u'NUMBER'))
                    elif u'-' in entity:
                        word_seperate = entity.split(u'-')
                        for word in word_seperate:
                            if word in time_word:
                                result_sentence.append((entity,u'NUMBER'))
                                break
                    elif entity in location_word:
                        result_sentence.append((entity,u'LOCATION'))
                    elif index == 0 and entity.lower() not in stopwords:
                        result_sentence.append((entity,u'ORGANIZATION'))
                    elif index != 0 and entity[0].isupper():
                        result_sentence.append((entity,u'ORGANIZATION'))
                    else:
                        result_sentence.append((entity,etype))
                elif entity in ['(',')']:
                    result_sentence.append((entity,u'O'))
                else:
                    result_sentence.append((entity,etype))
        result_sentences.append(result_sentence)
        
    return analyse_NER(result_sentences)

def extract_NER(parse_ner_sentence,mode):
    result = []
    if mode == 0:
        #PERSON
        for entity,etype in parse_ner_sentence:
            if etype == u'PERSON':
                result.append(entity)
    elif mode == 1:
        #NUMBER
        for entity,etype in parse_ner_sentence:
            if etype == u'NUMBER':
                result.append(entity)
    elif mode == 2:
        #LOCATION
        for entity,etype in parse_ner_sentence:
            if etype == u'LOCATION':
                result.append(entity)
    elif mode == 3:
        #ORGANIZATION
        for entity,etype in parse_ner_sentence:
            if etype == u'ORGANIZATION':
                result.append(entity)
    elif mode == 4:
        #YEAR
        for entity,etype in parse_ner_sentence:
            if etype == u'YEAR':
                result.append(entity)
    return result

def parse_token(token_sentence):
    result = []
    for index,word in enumerate(token_sentence):
        if index != 0 and index != (len(token_sentence)-1) and word == u'.':
            last_word = result[-1]
            last_word = last_word + u'.'
            result = result[:-1]
            result.append(last_word)
        else:
            result.append(word)
    return result    

In [21]:
def rank_rule_1(entity,query):
    #lower scores for content words also appear in the query
    count = 0
    length = len(entity)
    for word in entity:
        word = lemmatize(word.lower())
        if word not in stopwords:
            if word in query:
                count += 1
    if length == 0:
        score = 0
    else:
        score = 1 - float(count)/length
    return score

In [22]:
import string
def get_open_class_words(query):
    result = []
    for index in range(len(query)):
        if query[index] not in stopwords:
            if query[index] not in string.punctuation:
                result.append(query[index])
    return result

In [23]:
def rank_rule_3(answer_sentence,sentence,entity,query):
    #higher scores for closer distance between an entity and the headword
    #step 1: using a filter to extract "useful" open-class words
    results = get_open_class_words(query)
    sent = sentence
    original_sent = answer_sentence
    entity_loc = []
    query_loc = []
    for word in entity:
        if word in original_sent:
            entity_loc.append(original_sent.index(word))
    for q in results:
        if q in sent:
            query_loc.append(sent.index(q))
    min_dist = len(original_sent)
    if query_loc != []:
        for i in query_loc:
            for j in entity_loc:
                dist = abs(i - j)
                if dist < min_dist:
                    min_dist = dist
                    
    return 1 - float(min_dist)/len(original_sent)

In [24]:
def screen_out_answer_WHAT(result):
    answer_list = []
    for wtype, word in result:
        if wtype != 'J':
            answer_list.append(word)
    return answer_list

In [25]:
ids = []
answers = []
for index in range(len(test)):
    print index
    article = test[index]
    qas = article['qa']
    sentences = article['sentences']
    token_sentences = copy.deepcopy(sentences)
    for i in range(len(sentences)):
        token_sentences[i] = parse_token(nltk.word_tokenize(token_sentences[i]))
    ner_sentences = st.tag_sents(token_sentences)
    parse_ner_sentences = parse_NER(ner_sentences)
    for i in range(len(qas)):
        qa = qas[i]
        anss = []
        scs = []
        id = qa['id']
        for ii in range(4):
            if result_sentences_weight[index][i][ii] != 0:
                weight = float(result_sentences_weight[index][i][ii])/sum(result_sentences_weight[index][i])
            else:
                weight = 0
            answer_sentence = sentences[result_sentences[index][i][ii]]
            answer_sentence_id = result_sentences[index][i][ii]
            text_question = qa['question']
            result = get_continuous_chunks(text_question)

            if result != []:
                wtype,word = result[0]
                if wtype == 'NUMBER':
                    answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],1)
                    if answer_list == []:
                        answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],3)
                elif wtype == 'WHO':
                    answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],0)
                    if answer_list == []:
                        answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],2)
                        answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],3)
                elif wtype == 'WHERE':
                    answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],2)
                    if answer_list == []:
                        answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],0)
                        answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],3)
                elif wtype == 'WHEN':
                    answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],4)
                else:
                    #what, or other types
                    answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],3)
                    if answer_list == []:
                        answer_sentence = sentences[answer_sentence_id]
                        wpos,wresult = get_continuous_chunks_sentence(answer_sentence,0)
                        answer_list = screen_out_answer_WHAT(wresult)

            if answer_list != []:            
                query = copy.deepcopy(text_question)
                query = nltk.word_tokenize(query)
                #token the answer sentence and copy it for further usage
                answer_sentence = nltk.word_tokenize(answer_sentence)
                sentence = copy.deepcopy(answer_sentence)

                for query_index in range(len(query)):
                    query[query_index] = lemmatize(query[query_index].lower())
                for sent_index in range(len(sentence)):
                    sentence[sent_index] = lemmatize(sentence[sent_index].lower())

                scores_1 = []
                scores_3 = []
                scores = []
                
                for entity in answer_list:
                    entity = nltk.word_tokenize(entity) 
                    score1 = rank_rule_1(entity,query)
                    scores_1.append(score1)
                    #answer_sentence is the original version and sentence is preprocessed
                    score3 = rank_rule_3(answer_sentence,sentence,entity,query)
                    scores_3.append(score3)
                    w1 = 0.2
                    w3 = 1 - w1
                    if score1 == 0:
                        score3 = 0
                    total = w1 * score1 + w3 * score3
                    scores.append(total)
                answer = answer_list[scores.index(max(scores))]
                sc = max(scores)*weight

            else:
                answer = answer_sentence
                sc = 0
            anss.append(answer)
            scs.append(sc)
        ans = anss[scs.index(max(scs))]
        
        ids.append(id)
        answers.append(ans)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [26]:
for i in range(len(answers)):
    answers[i] = answers[i].replace(',','-COMMA-')
    answers[i] = answers[i].replace('"','')

In [27]:
def output_result(filename):

    predictions_file = open(filename, "wb")
    open_file_object = csv.writer(predictions_file)
    open_file_object.writerow(["id","answer"])
    for i in range(len(answers)):
        open_file_object.writerow([ids[i], answers[i].encode("utf-8")])
    predictions_file.close()

In [28]:
output_result("result_enhance_3.csv")

In [29]:
print answers[144]

3-COMMA-300
